# Git Hooks Examples with Runnable Code

This notebook contains examples of different Git hooks that can be useful to use for automating tasks in your Git workflow. Each example includes runnable code snippets. The [pre-commit](#1-pre-commit-hook) and [post-commit](#3-post-commit-hook) hooks have been implemented into this repository and are live and running. The other hook types are here just as examples, they are not saved as live scripts in this repository.

## Table of Contents

[Installing the Hooks](#installing-the-hooks)
1. [Pre-commit Hook](#1-pre-commit-hook)
2. [Commit-msg Hook](#2-commit-msg-hook)
3. [Post-commit Hook](#3-post-commit-hook)
4. [Pre-receive Hook](#4-pre-receive-hook)
5. [Update Hook](#5-update-hook)
6. [Post-receive Hook](#6-post-receive-hook)

## Installing the Hooks

The following describes the manual method. This is, however, necessary only for edits of a pre-commit script file. For other hooks, like the post-commit, we have implemented an automatic installation of the hooks in the install-hooks.sh file (in the hooks-accessories folder). This installation file is called before every commit from the pre-commit hook, to update the hook scripts.

For the installation, one needs to copy the hooks files from the 'hooks/' folder to the '.git/hooks/' folder. This can be done by executing the 'install-hooks.sh' Bash script. When changing from Windows to Unix systems, the following line ending conversion might be necessary first, though. In general, most errors occured to us when wrong line endings were applied to bash scripts, and they were attempted to be run from mismatched systems.

Open Git Bash, and type in:

```bash
# For Unix systems:
dos2unix hooks-accessories/install-hooks.sh

# For Windows systems:
# unix2dos .git/hooks/post-commit

# And run the install script:
bash hooks-accessories/install-hooks.sh
```

To verify the installation has been successful, navigate to the '.git/hooks/' folder and check for our hook files, for example if the post-commit file is present.

The following should also be done automatically by the install-hooks.sh. In case the automatic script doesn't work from the pre-commit, run it manually. On some combinations of systems or environments, after the hooks are copied, it might be necessary to run the following command to convert the Windows-style line endings (CRLF) to Unix-style line endings (LF). Otherwise the script won't execute properly, or not at all. The hooks need to be copied to the hooks folder and used before the git commits and converts the line endings itself. The line endings of the scripts need to match the environment that we're commiting from. So, for example, if the hook script was edited on a Windows system, copied to the '.git/hooks/' folder, but not treated with line endings for Linux, and then a Windows Subsystem for Linux (WSL) terminal is used to commit the files, it will throw an error about the hooks. Because the WSL expects LF endings, while the scripts have CRLF endings.

Run the following so the hooks can be executed on Unix-like systems:
```bash
# Run in a (WSL) Bash terminal
dos2unix .git/hooks/pre-commit
dos2unix .git/hooks/post-commit
```

Or, alternatively, the reverse scenario could happen, a Windows user needing to use a script that has LF endings and Git not converting them, yet.
Run the following so the hooks can be executed on Windows-style systems:
```bash
# Run in a Git Bash terminal
unix2dos .git/hooks/pre-commit
unix2dos .git/hooks/post-commit
```
This could be automated in a pre-commit hook, for the other hooks. But we're not sure yet, how to solve the automation for the pre-commit hook itself.

### Environment Set-up

For some hooks, there are a few prerequisites before they work. The local environment has to be set up correctly.

The pre-commit hook requires:
- flake8, pytest, coverage (available through pip install)

The post-commit hook requires a rather complicated set-up:
- git-crypt
    - `sudo apt install git-crypt`
    - access to the encrypted file can be gained by using the .gpg key for all collaborators
- GPG
    - `sudo apt install gnupg`
    - collaborators with GPG keys will use their own private key to unlock the repository (after you add their public GPG key using `git-crypt add-gpg-user <key-id>`).

### Our Repo Live Hooks Output

The following picture shows an example output when making a git commit from our local environment on this repo. A pre- and post-commit hooks are used successfully.

![Hooks live run output](./hooks-accessories/Hooks_live_run.JPG "Hooks live run output")

## 1. Pre-commit Hook

The `pre-commit` hook is a hook that runs before a commit is made. We can use it to check code quality, run tests, or enforce coding standards.

### Example Description

**Scenario:** A Python project is using Continuous Integration (CI) to run tests and deploy code. Developers want to ensure that code pushed to the repository passes all tests before it reaches the CI stage.

**Pre-Commit Hook Implementation:**

 - Unit Tests: Automatically run all unit tests using pytest.
 - Code Coverage: Check code coverage to ensure that enough of the code is being tested.
 - Static Analysis: Use flake8 to enforce coding standards.

### How to Implement:

The pre-commit hook is already implemented in this repo. But in theory, if someone needed to make a new pre-commit on their system manually, here are the steps:

1. Navigate to the hooks directory: `cd .git/hooks`
2. Create a new file: `touch pre-commit`
3. Make it executable: `chmod +x pre-commit`
4. Add the below code to the `pre-commit` file.

### Hook Install
We have implemented an automatic install of the hooks. We call the install-hooks.sh from the pre-commit script to automatically update the scripts before every commit. This is one useful example of the pre-commit hook usage.

### Code Checks
For code quality checks, we show and implement the following pre-commit hook examples. If either of them fails, it aborts the commit, in order for the code quality and tests to be on par before the changes are committed.

- Flake8 - a detector of code style issues and programming errors. Examples of such issues is PEP8 styling violations, or undefined programming names in the code.

- Pytest - looks for any test files in the repo that adhere to naming conventions like test_*.py or *_test.py, and runs them.

- Code coverage - tracks how much of the code is covered by unit tests of Pytest.

Flake8, Pytest, and coverage need to be installed in the environment, from which we are making the commits.

### Code - hook script:

```bash
#!/bin/bash
# A Git pre-commit hook example.
# This is a live working script, implemented in the repo.


#---------------------#
#----- Flag check ----#
#---------------------#

# This check is necessary to not run pre-commit on additional amend commits that happen in the post-commit hook.
# Check if the commit has already been amended by looking for a flag file.
FILE_FLAG=".amend-flag"

if [ -f "$FILE_FLAG" ]; then
    # If the flag file exists, exit to avoid an infinite loop, but don't remove the file, that is done in the post-commit hook
    echo "Exiting the re-triggered pre-commit hook to avoid loop."
    exit 0
fi


#######################
####### Messages ######
#######################

echo "HOOK - PRE-COMMIT - START:"
echo "Your commit is about to happen. Running the pre-commit hooks..."


#######################
#### Hooks install ####
#######################

# Run install-hooks.sh to update Git hooks and adjust line endings
echo "Running install-hooks.sh to update hooks and adjust their line endings..."
# If the following doesn't run, do a "dos2unix hooks-accessories/install-hooks.sh" manually in terminal for Unix-systems
bash hooks-accessories/install-hooks.sh
INSTALL_HOOKS_EXIT_CODE=$?
if [ $INSTALL_HOOKS_EXIT_CODE -ne 0 ]; then
  echo "Failed to run install-hooks.sh. Please check the script and try again."
  echo "Aborting the commit."
  exit 1
fi


#######################
##### Code checks #####
#######################

# Flake8, style guide enforcement
echo "Running Flake8, a detector of code style issues and programming errors..."
flake8 . --exclude=.venv # current directory, excluding the python .venv folder
if [ $? -ne 0 ]; then
  echo "Code issues detected according to Flake8. Please fix them before committing."
  echo "Aborting the commit."
  exit 1 # stop the commit
fi

# Pytest, unit tests
echo "Running Pytest, unit tests in the repo..."
pytest
PYTEST_EXIT_CODE=$?
if [ $PYTEST_EXIT_CODE -eq 0 ]; then
  echo "Pytest completed successfully."
elif [ $PYTEST_EXIT_CODE -eq 5 ]; then
  echo "No Pytest test was found. Proceeding with the commit."
else
  echo "Pytest unit tests failed with exit code $PYTEST_EXIT_CODE. Please fix the issues before committing."
  echo "Aborting the commit."
  exit 1
fi

# Code coverage check
echo "Checking code coverage of Pytest..."
coverage run -m pytest # the --no-data-file is somehow buggy for us, so omitted, and removing the report manually later
COVERAGE_EXIT_CODE=$?

if [ $COVERAGE_EXIT_CODE -eq 0 ]; then
  echo "Coverage check completed successfully."
elif [ $COVERAGE_EXIT_CODE -eq 5 ]; then
  echo "No data was collected. Proceeding with the commit."
else
  echo "Coverage check failed with exit code $COVERAGE_EXIT_CODE. Please ensure sufficient coverage before committing."
  echo "Aborting the commit."
  exit 1
fi

# Clean up: Remove the .coverage file if it exists
if [ -f ".coverage" ]; then
  rm .coverage
  echo ".coverage file removed after coverage check."
fi


# Display the end message
echo "HOOK - PRE-COMMIT - END"
```

## 2. Commit-msg Hook

The `commit-msg` hook is triggered after the commit message is entered but before the commit is finalized. It can be used to enforce a specific commit message format.

### Example Description

**Scenario:** A software development team wants to maintain a consistent format for commit messages, particularly for tracking issues related to project management tools like Jira. They want to ensure that all commit messages start with a specific ticket number format (e.g., `JIRA-123: Description of the change`).

**Pre-commit Hook Implementation:**

- **Enforce Commit message Format**: Check that the commit message starts with a JIRA ticket number.

### How to Implement:
1. Navigate to the hooks directory: `cd .git/hooks`
2. Create a new file: `touch commit-msg`
3. Make it executable: `chmod +x commit-msg`
4. Add the below code to the `commit-msg` file.

Alternatively, if the pre-commit hook is already working in your environment, you could place this script into the Git root folder `hooks`, from where the pre-commit would automatically copy it to the `.git/hooks` folder. Some edits to the `install-hooks.sh` install script might be necessary, though, to include this hook into line ending conversions etc.

### Code - hook script:


```bash
# This is only an example commit-msg hook to enforce message format - addittional changes as testing the functionality need to be made first on specific cases.
#!/bin/sh

# Get the commit message file path from the first argument
commit_msg_file="$1"

# Check if the commit message file was provided
if [ -z "$commit_msg_file" ]; then
  echo "Error: No commit message file provided."
  echo "Usage: commit-msg <commit-message-file>"
  exit 1
fi

# Check if the commit message starts with "JIRA-123: "
if ! grep -qE "^JIRA-[0-9]+: " "$commit_msg_file"; then
  echo "Error: Commit message must start with a JIRA ticket number, e.g., 'JIRA-123: Fix bug.'"
  exit 1
fi

# If the format is correct, print a success message
echo "Commit message format is correct."
```

## 3. Post-commit Hook

The `post-commit` hook is a hook that runs after a commit has been made. It can do various things after the commit, ranging from simply displaying a custom message if the commit was successful, running any post-commit tests and changes, logging, automatically pushing, notifications, etc. In this repository, we have a working post-commit script that displays a few messages after the commit. It also updates a log file with a new entry, and a notification is sent to Slack and Discord. An automatic push is commented out for safety purposes, as we deemed it risky.

For the notifications, we use webhook URLs generated from Slack and Discord. In order not to publicly share these in this repository, we git-crypt the hooks-accessories/secret.txt file where we store the variables SLACK_WEBHOOK_URL and DISCORD_WEBHOOK_URL. Git-crypt will automatically encrypt files that need to be kept secret, such as our secret.txt. A working environment with correctly set up git-crypt and GPG keys is needed for encrypting and decrypting this file. The git-crypt method is more complicated on Windows, since one has to run the git commands from the WSL environment where the git-crypt is installed (a global Windows installation has questionable support). A drawback of using git-crypt in a repo is that when one does not have a functioning git-crypt local environment, no commits are possible in the git repo, unless one deletes the .git-crypt folder from the repo and maybe comment out the .pitattributes git-crypt line.

### Example description

**Scenario:** A development team wants to keep everyone informed about new commits to the repository. They decide to send a notification to a Slack channel every time a commit is made.

**Post-Commit Hook Implementation:**

- **Notify Team**: Send a message to a Slack channel with details of the latest commit.

### How to Implement:

The post-commit hook is already implemented in this repo. But in theory, if someone needed to make a new post-commit on their system manually, here are the steps:

1. Navigate to the hooks directory: `cd .git/hooks`
2. Create a new file: `touch post-commit`
3. Make it executable: `chmod +x post-commit`
4. Add the below code to the `post-commit` file.

### Our Repo Live Hooks Output

Following is a picture of the commit-log.txt log file, and the Slack and Discord channels with notifications from our repo commits.

![Hooks live run log file](./hooks-accessories/Hooks_live_log.JPG "Hooks live run log file")

![Hooks live run Slack notification](./hooks-accessories/Hooks_live_notif_slack.JPG "Hooks live run Slack notification")

![Hooks live run Discord notification](./hooks-accessories/Hooks_live_notif_discord.JPG "Hooks live run Discord notification")

### Code - hook script:

```bash
#!/bin/bash
# A Git post-commit hook that displays messages and additional information.
# This is a live working script, implemented in the repo.


#---------------------#
#----- Flag check ----#
#---------------------#

# This check is necessary for our Logging part of code, where we amend the commit.
# Without this flag check, the post-commit hook would enter an infinite loop.
# Check if the commit has already been amended by looking for a flag file.
FILE_FLAG=".amend-flag"

if [ -f "$FILE_FLAG" ]; then
    # If the flag file exists, remove it and exit to avoid an infinite loop
    rm -f "$FILE_FLAG"
    echo "Exiting the re-triggered hook to avoid loop."
    exit 0
fi


#######################
####### Messages ######
#######################

echo "HOOK - POST-COMMIT - START:"
echo "Your commit succeeded. This is a message from IES squad."


#######################
####### Logging #######
#######################

echo "Logging commit details..."

# Log file location
FILE_LOG="hooks-accessories/commit-log.txt"

# Prepare the commit details
COMMIT_HASH=$(git log -1 --pretty=format:"%H")
COMMIT_MESSAGE=$(git log -1 --pretty=format:"%s")
COMMIT_AUTHOR=$(git log -1 --pretty=format:"%an")
COMMIT_DATE=$(git log -1 --pretty=format:"%ad")

# Append info to the log file
echo "------------------------------------------------" >> $FILE_LOG
echo "Commit hash: $COMMIT_HASH" >> $FILE_LOG
echo "Author: $COMMIT_AUTHOR" >> $FILE_LOG
echo "Date: $COMMIT_DATE" >> $FILE_LOG
echo "Message: $COMMIT_MESSAGE" >> $FILE_LOG
echo "" >> $FILE_LOG

echo "Done appending commit details to $FILE_LOG."

# Stage the log file so it gets included in the commit
git add $FILE_LOG

# Create the flag file to prevent re-triggering the hook
touch "$FILE_FLAG"

# Amend the last commit to include the updated log file, without changing the commit message
git commit --amend --no-edit

# Remove the flag file after the commit is amended
rm -f "$FLAG_FILE"

echo "Commit log updated and last commit amended."


#######################
#### Notifications ####
#######################

echo "Running post-commit notifications..."

#--- URL retrieval ---#
    FILE_SECRET="hooks-accessories/secret.txt"

    # Check if the secret file exists and is readable
    if [ -f "$FILE_SECRET" ]; then
        # Read the secret URLs from the file
        source "$FILE_SECRET"
    else
        echo "Error: secret.txt not found or not readable."
        exit 1
    fi

# Webhook URLs retrieved from secret.txt
if [[ -n "$SLACK_WEBHOOK_URL" && -n "$DISCORD_WEBHOOK_URL" ]]; then
    echo "Webhook URLs successfully retrieved."

### Slack
    # Slack webhook URL (generated from Slack/Apps/Incoming WebHooks)
    # Retrieved from the git-crypted secret.txt file
        # SLACK_WEBHOOK_URL="https://hooks.slack.com/services/your/slack/webhook"

    # Create a JSON string
    SLACK_PAYLOAD="{
        \"text\": \"Notification - a new commit\nCommit hash: <$COMMIT_HASH>\nAuthor: *$COMMIT_AUTHOR*\nDate: $COMMIT_DATE\nMessage: $COMMIT_MESSAGE\"
    }"
    
    # Trim any leading or trailing whitespace from the SLACK_WEBHOOK_URL.
    # This is necessary when the variable is on the first line of the secret.txt file.
    SLACK_WEBHOOK_URL=$(echo "$SLACK_WEBHOOK_URL" | tr -d '\r' | xargs)

    # Send the string to Slack as a payload and capture the response
    RESPONSE=$(curl -s -X POST -H 'Content-type: application/json' --data "$SLACK_PAYLOAD" $SLACK_WEBHOOK_URL)

    # Check if the response from Slack contains "ok"
    if [[ "$RESPONSE" == "ok" ]]; then
        echo "Slack notification sent."
    else
        echo "Slack notification unsuccessful. Response: $RESPONSE"
    fi
    
### Discord
    # Discord webhook URL (generated from Discord/Channel/Integrations/Webhook)
    # Retrieved from the git-crypted secret.txt file
        # DISCORD_WEBHOOK_URL="https://discord.com/api/webhooks/your/discord/webhook"

    # Create JSON payload
    DISCORD_PAYLOAD="{
        \"content\": \"Notification - a new commit\nCommit hash: <$COMMIT_HASH>\nAuthor: **$COMMIT_AUTHOR**\nDate: $COMMIT_DATE\nMessage: $COMMIT_MESSAGE\"
    }"

    # Trim any leading or trailing whitespace from the SLACK_WEBHOOK_URL
    # This is necessary when the variable is on the first line of the secret.txt file.    
    DISCORD_WEBHOOK_URL=$(echo "$DISCORD_WEBHOOK_URL" | tr -d '\r' | xargs)

    # Send the string to Discord as a payload and capture the HTTP status code
    HTTP_STATUS=$(curl -s -o /dev/null -w "%{http_code}" -X POST -H 'Content-type: application/json' --data "$DISCORD_PAYLOAD" $DISCORD_WEBHOOK_URL)

    # Check if the status code is 2xx (success)
    if [[ "$HTTP_STATUS" -ge 200 && "$HTTP_STATUS" -lt 300 ]]; then
        echo "Discord notification sent."
    else
        echo "Discord notification unsuccessful. Status code: $HTTP_STATUS"
    fi

else
    echo "Error: Webhook URLs not found in secret.txt."
fi

echo "Done sending notifications."


#######################
######## Push #########
#######################

# Note: careful with this, more dangerous. Tested, works, but commenting it out for safer practices.
# Optionally, can automatically push the commit to the remote repository.
    # git push --force-with-lease origin $(git branch --show-current)


# Display the end message
echo "HOOK - POST-COMMIT - END"
```

## 4. Pre-receive Hook

The `pre-receive` hook is a server-side hook that runs before any references are updated. It can be used to enforce policies on incoming pushes.

### Example Description

**Scenario:** A team wants to prevent direct pushes to the main branch of their repository to ensure that all changes are reviewed via pull requests. This helps maintain code quality and facilitates collaboration.

**Pre-receive Hook Implementation:**

- **Block Pushes**: Disallow direct pushes to the main branch.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch pre-receive`
3. Make it executable: `chmod +x pre-receive`
4. Add the below code to the `pre-receive` file.

### Code - hook script:

```bash
# This is only an example commit-msg hook to enforce message format - addittional changes as testing the functionality need to be made first on specific cases.
#!/bin/sh

# Read incoming pushes
while read oldrev newrev refname; do
  # Check if the push is to the main branch
  if [ "$refname" = "refs/heads/main" ]; then
    echo "Error: Direct pushes to the main branch are not allowed."
    echo "Please create a pull request instead."
    exit 1
  fi
done

# If no errors, allow the push to proceed
echo "Push accepted."
```

## 5. Update Hook

The `update` hook runs before a reference is updated. It is useful for enforcing specific policies on branch updates.

### Example Description

**Scenario:** A development team wants to prevent force pushes to the main branch to protect the commit history. This ensures that all changes are properly reviewed and merged through pull requests.

**Update Hook Implementation:**

- **Reject Force Pushes**: Disallow force pushes to the `main` branch while allowing normal pushes.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch update`
3. Make it executable: `chmod +x update`
4. Add the below code to the `update` file.

### Code - hook script:

```bash
# This is only an example commit-msg hook to enforce message format - addittional changes as testing the functionality need to be made first on specific cases.
#!/bin/sh

# Read incoming pushes
while read oldrev newrev refname; do
  # Check if the push is to the main branch
  if [ "$refname" = "refs/heads/main" ]; then
    # Check if the old revision is not empty (not a new branch)
    if [ "$oldrev" != "0000000000000000000000000000000000000000" ]; then
      echo "Error: Force pushes to 'main' branch are not allowed."
      exit 1
    fi
  fi
done

# Allow the push to proceed if checks pass
echo "Push accepted."

# Exit with a success status
exit 0
```

## 6. Post-receive Hook

The `post-receive` hook runs after a push has been received and processed. It's commonly used for deployment tasks.

### Example Description

**Scenario:** A web application is hosted on a server, and the development team wants to automate the deployment process whenever new code is pushed to the repository. This ensures that the latest code is always live without requiring manual intervention.

**Post-receive Hook Implementation:**

- **Automatic Deployment**: Automatically deploy the latest code to the production environment.

### How to Implement:
1. Navigate to the hooks directory on the server: `.git/hooks`
2. Create a new file: `touch post-receive`
3. Make it executable: `chmod +x post-receive`
4. Add the below code to the `post-receive` file.

### Code - hook script:

```bash
# This is only an example commit-msg hook to enforce message format - addittional changes as testing the functionality need to be made first on specific cases.
#!/bin/sh

# Deploy the latest code to the specified working tree
if git --work-tree="$WORK_TREE" --git-dir="$GIT_DIR" checkout -f; then
  echo "Deployment successful: Latest code has been deployed to $WORK_TREE."
else
  echo "Error: Deployment failed."
  exit 1
fi
```